In [ ]:
!pip install -U featuretools
!pip install nvcc4jupyter
#!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [ ]:
%%writefile main.cu
#include <stdio.h>
#include <iostream>
#include <stdexcept>


class BinaryImage {

  public:
    BinaryImage(int num_rows, int num_cols);
    void set(int value, int row, int column);
    __device__ void gpu_set(int value, int row, int column);
    void print(void);
    void allocate_space_on_gpu(void);
    void move_image_to_gpu(void);
    void move_image_from_gpu_to_host(void);
    ~BinaryImage(void);
    int num_rows;
    int num_cols;
    int* data;
    int* gpu_data;

  private:
    int index(int row, int column);
    int size();
    int __device__ device_index(int row, int column);
};

BinaryImage::BinaryImage(int num_rows, int num_cols): num_rows(num_rows), num_cols(num_cols) {

  // Allocate space for the int*'s
  data = (int*) malloc(size());

  // Create the empty image.
  for (int i = 0; i < num_rows; i++) {
    for (int j = 0; j < num_cols; j++) {
      data[index(i, j)] = 0;
    }
  }

  // Pre-allocate space on the GPU.
  allocate_space_on_gpu();
}

BinaryImage::~BinaryImage(void) {
  // Free the memory on the gpu.
  // Free the memory on the cpu.
}

int BinaryImage::index(int row, int column) {
  return (row * num_cols) + column;
}

__device__ int BinaryImage::device_index(int row, int column) {
  return (row * num_cols) + column;
}

void BinaryImage::set(int value, int row, int column) {

  if ((row < 0) || (row >= num_rows) || (column < 0) || (column >= num_cols)){
    throw std::invalid_argument("Illegal image coordinates.");
  }
  else if (value != 0 && value != 1) {
    throw std::invalid_argument("Illegal pixel value.");
  }

  data[index(row, column)] = value;
}

__device__ void BinaryImage::gpu_set(int value, int row, int column) {

  gpu_data[device_index(row, column)] = value;
}

void BinaryImage::print() {

  for (int i = 0; i < num_rows; i++) {
    for (int j = 0; j < num_cols; j++) {
      std::cout << data[index(i, j)] << " ";
    }
    std::cout << std::endl;
  }

}

void BinaryImage::allocate_space_on_gpu() {

  // Allocate space on the GPU.
  cudaMalloc((void**)&gpu_data, size());
}

void BinaryImage::move_image_to_gpu(void) {

  cudaMemcpy(gpu_data, data, size(), cudaMemcpyHostToDevice);
}

void BinaryImage::move_image_from_gpu_to_host(void) {

    cudaMemcpy(data, gpu_data, size(), cudaMemcpyDeviceToHost);
}

int BinaryImage::size() {
  return (int)(sizeof(int) * num_rows * num_cols);
}

struct Complex {

  float real;
  float imaginary;

  __device__ Complex(float real, float imaginary): real(real), imaginary(imaginary) {};

  __device__ float magnitude(void) {
    return (real * real) + (imaginary * imaginary);
  }

  __device__ Complex operator*(const Complex& right) {
    float result_real = (real * right.real) - (imaginary * right.imaginary);
    float result_imaginary = (imaginary * right.real) + (real * right.imaginary);
    return Complex(result_real, result_imaginary);
  }

  __device__ Complex operator+(const Complex& right) {
    return Complex(real + right.real, imaginary + right.imaginary);
  }


};


__device__ bool julia_set(int* image, int row, int column, int num_rows, int num_cols) {

  Complex c = Complex(-0.8, 0.156);
  int scale = 1.5;
  int jx = scale * (float)((num_cols/2) - row)/((num_cols/2));
  int jy = scale * (float)((num_rows/2) - column)/((num_rows/2));
  Complex a = Complex(jx, jy);

  int max_iterations = 200;
  int iteration = 0;

  while (iteration < max_iterations) {

    a = (a * a) + c;

    if (a.magnitude() > 2000) {
      return false;
    }

    iteration += 1;
  }


  return true;
}


__global__ void myKernel(int* image, int num_rows, int num_cols) {

  int row = blockIdx.x;
  int column = threadIdx.x;
  int index = (row * num_cols) + column;

  if (julia_set(image, row, column, num_rows, num_cols)) {
    image[index] = 1;
  }
  else {
    image[index] = 0;
  }
}

int main() {

  int num_rows = 40;
  int num_cols = 40;
  BinaryImage image = BinaryImage(num_rows, num_cols);

  // Move data to the GPU.
  image.move_image_to_gpu();

  // Run the kernel.
  myKernel<<<num_rows, num_cols>>>(image.gpu_data, num_rows, num_cols);

  // Move data back from the GPU.
  image.move_image_from_gpu_to_host();

  image.print();

  return 0;
}

Overwriting main.cu


In [ ]:
%%script bash
nvcc main.cu -o julia_set

In [ ]:
%%script bash
./julia_set

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 